# Pubsub Demo

"Asynchronous Stampers"

In [ ]:
import pandas as pd

from pubsub_demo.client import StampsApi

api = StampsApi()

columns = ["entry_id", "request_id", "series_num", "request_type", "stamps", "runtime_ms"]


In [ ]:
# let's start with a clean DB
api.delete_stamps()

### Short Path
Messages dispatched by service "A" need to be stamped by service "B" only, before final collection by service "A".

In [ ]:
# Issue a request for stamps
request_id = api.create_stamps(type="short", num=1000)
request_id 

In [ ]:
# Fetch the results
results = api.retrieve_stamps(id=request_id)
df = pd.DataFrame(results, columns=columns)
df.sort_values(by=["request_id", "series_num"]).head(10)

In [ ]:
# Length should match what we requestd
# (NOTE: error handling must still be added to prevent the occasional lost message)
len(df)

In [ ]:
# How long did it take on average?
df["runtime_ms"].mean()

In [ ]:
# Load balancing ratio
num_b1 = len(df[df["stamps"]=="B1"])
num_b2 = len(df[df["stamps"]=="B2"])
ratio = num_b1/(num_b1 + num_b2)
ratio

### Long Path
Messages dispatched by service "A" need to be stamped by service "B" and then by service "C" (in that order), before final collection by service "A".

In [ ]:
# Issue a request for stamps
request_id = api.create_stamps(type="long", num=100)
request_id 

In [ ]:
# Fetch the results
results = api.retrieve_stamps(id=request_id)
df = pd.DataFrame(results, columns=columns)
df.sort_values(by=["request_id", "series_num"]).head(10)

In [ ]:
# Length should match what we requestd
len(df)

In [ ]:
# How long did it take on average?
df["runtime_ms"].mean()

In [ ]:
# Load balancing ratio
num_b1 = len(df[df["stamps"]=="B1-C"])
num_b2 = len(df[df["stamps"]=="B2-C"])
ratio = num_b1/(num_b1 + num_b2)
ratio